<a href="https://colab.research.google.com/github/wko1014/GAN_Study/blob/main/notes/Deep_Convolutional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import APIs
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Download MNIST Dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/data/", one_hot=True)

# Define Hyperparameters and Placehoders
epochs = 100
batch_size = 100
learning_rate = 1e-3
n_noise = 100
n_image = 28 * 28

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Z = tf.placeholder(tf.float32, [None, n_noise])
is_training = tf.placeholder(tf.bool)

Extracting ./MNIST_data/data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/data/t10k-labels-idx1-ubyte.gz


In [ ]:
# Define Moduels
def leaky_relu(x, leak=0.2):
    return tf.maximum(x, x * leak)

In [ ]:
# Define Generator (George)
def generator(noise):
    with tf.variable_scope("george"):
        layer1 = tf.layers.dense(noise, 128*7*7)
        layer1 = tf.reshape(layer1, [-1, 7, 7, 128])
        layer1 = tf.nn.relu(tf.layers.batch_normalization(layer1, training=is_training))

        layer2 = tf.layers.conv2d_transpose(layer1, 64, [5, 5], strides=(2, 2), padding="SAME")
        layer2 = tf.nn.relu(tf.layers.batch_normalization(layer2, training=is_training))

        layer3 = tf.layers.conv2d_transpose(layer2, 32, [5, 5], strides=(2, 2), padding="SAME")
        layer3 = tf.nn.relu(tf.layers.batch_normalization(layer3, training=is_training))

        output = tf.layers.conv2d_transpose(layer3, 1, [5, 5], strides=(1, 1), padding="SAME")
        output = tf.nn.tanh(output)
    return output


In [ ]:
# Define Discriminator (Danielle)
def discriminator(image, reuse=False):
    with tf.variable_scope("danielle", reuse=reuse):
        layer1 = tf.layers.conv2d(image, 32, [5, 5], strides=(2, 2), padding="SAME")
        layer1 = leaky_relu(layer1)

        layer2 = tf.layers.conv2d(layer1, 64, [5, 5], strides=(2, 2), padding="SAME")
        layer2 = leaky_relu(tf.layers.batch_normalization(layer2, training=is_training))

        layer3 = tf.layers.conv2d(layer2, 128, [5, 5], strides=(2, 2), padding="SAME")
        layer3 = leaky_relu(tf.layers.batch_normalization(layer3, training=is_training))

        output = tf.contrib.layers.flatten(layer3)
        output = tf.layers.dense(output, 1, activation=None)
        output = tf.nn.sigmoid(output)
    return output

In [ ]:
# Sample Random Noise Vector
def get_noise(batch_size, n_noise):
  return np.random.uniform(-1., 1., size=(batch_size, n_noise))

def get_moving_noise(batch_size, n_noise):
    assert batch_size > 0
    noise_list = []
    base_noise = np.random.uniform(-1., 1., size=[n_noise])
    end_noise = np.random.uniform(-1., 1., size=[n_noise])

    step = (end_noise - base_noise) / batch_size
    noise = np.copy(base_noise)
    for _ in range(batch_size - 1):
        noise_list.append(noise)
        noise = noise + step
    noise_list.append(end_noise)

    return noise_list

# George Generates Counterfeits
counterfeits = generator(noise=Z)

# Danielle Discriminates Real Bills and Counterfeits
D_real = discriminator(X)
D_counterfeits = discriminator(counterfeits, reuse=True)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.flatten instead.


In [ ]:
# GANs Objective Function
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_counterfeits))
loss_G = tf.reduce_mean(tf.log(D_counterfeits))

# Collect George's and Danielle's Variables
D_var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="danielle")
G_var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="george")

# Optimization using Adam Optimizer
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [ ]:
# Nerual Networks Training
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_var_D, loss_var_G = 0, 0

for epoch in range(epochs):
    for batch in range(total_batch):
        batch_x, _ = mnist.train.next_batch(batch_size)
        batch_x = np.reshape(batch_x, newshape=[-1, 28, 28, 1])
        noise = get_noise(batch_size, n_noise)

        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_x, Z: noise, is_training:True})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise, is_training:True})

    print("Epoch:", "%04d" % epoch, "D loss: {:.4}".format(loss_val_D), "G loss: {:.4}".format(loss_val_G))


    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        generated_samples = sess.run(counterfeits, feed_dict={Z: noise, is_training:False})
        moving_noise = get_moving_noise(sample_size, n_noise)
        moving_samples = sess.run(counterfeits, feed_dict={Z: moving_noise, is_training:False})

        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            ax[0][i].imshow(np.reshape(generated_samples[i], (28, 28)))
            ax[1][i].imshow(np.reshape(moving_samples[i], (28, 28)))

        plt.savefig("./{}.png".format(str(epoch).zfill(3)), bbox_inches="tight")
        plt.close(fig)

print("Optimization Complete!")

In [ ]:
import matplotlib.image as mpimg

img = mpimg.imread("./099.png")
plt.imshow(img)
plt.show()